In [3]:
import duckdb as duckdb


In [4]:
con = duckdb.connect(database="/Users/franciscobrito/workspaceGD/gerador-dados-teste/scripts/dados_duckdb.db", read_only=False)

In [9]:
resultado = con.execute("""
                        SELECT * 
                        FROM (
                            SELECT *, ROW_NUMBER() OVER (PARTITION BY NARBR ORDER BY data_ingestao DESC) as row 
                            FROM bronze_Z0019
                            WHERE data_ingestao >= '2025-01-11'
                        ) 
                        WHERE row = 1 
                        """).fetchdf()

resultado.head(10)


,NARBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao,row
0,10004,SERRA,BT50,100,200,z0019_2.csv,2026-01-04 08:54:48.921266,1
1,10003,PREGO,BT10,100,50,z0019_1.csv,2026-01-04 08:56:49.883106,1
2,10001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-01-04 08:56:49.883106,1
3,10002,MATELO,BT50,100,1500,z0019_1.csv,2026-01-04 08:56:49.883106,1
4,10005,MACHADO,BT50,100,100,z0019_2.csv,2026-01-04 08:54:48.921266,1


In [10]:
resultado_final = resultado.drop(columns=['nome_arquivo', 'data_ingestao', 'row'])
resultado_final = resultado_final.rename(columns = {'NARBR': 'id'})
resultado_final = resultado_final.rename(columns = {'MAKTX': 'nm_produto'})
resultado_final = resultado_final.rename(columns = {'WERKS': 'id_categgoria'})
resultado_final = resultado_final.rename(columns = {'MAINS': 'id_fornecedor'})
resultado_final = resultado_final.rename(columns = {'LABST': 'vl_preco'})
resultado_final.head(10)

,id,nm_produto,id_categgoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200
1,10003,PREGO,BT10,100,50
2,10001,PARAFUSO,BT10,100,100
3,10002,MATELO,BT50,100,1500
4,10005,MACHADO,BT50,100,100


In [11]:
resultado_final2 = resultado_final
resultado_final2 = resultado_final2.astype(
    {
        'id': int, 
        'nm_produto': str, 
        'id_categgoria': str, 
        'id_fornecedor': int, 
        'vl_preco': float
     })
resultado_final2.dtypes
#resultado_final2.head(10)

id                 int64
nm_produto        object
id_categgoria     object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [6]:
con.execute("""
CREATE TABLE IF NOT EXISTS produtos (
            id BIGINT,
            nm_produto TEXT,
            id_categoria TEXT,
            id_fornecedor BIGINT,
            vl_preco FLOAT
            )
""");

In [12]:
con.execute("insert into produtos select * from resultado_final2")

In [13]:
df_resultado = con.execute("SELECT * FROM produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10003,PREGO,BT10,100,50.0
2,10001,PARAFUSO,BT10,100,100.0
3,10002,MATELO,BT50,100,1500.0
4,10005,MACHADO,BT50,100,100.0


In [14]:
con.close()